## Pip

In [1]:
%pip install  pandas
#python -m venv <your venv name>
#source <your venv name>/bin/activate

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pickle-mixin

  Using cached pickle_mixin-1.0.2-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [53]:
pip install tqdm


Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install vk
%pip install pandas
%pip install requests
%pip install vk-api

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached vk_api-11.9.9-py3-none-any.whl.metadata (2.0 kB)
Using cached vk_api-11.9.9-py3-none-any.whl (48 kB)
Note: you may need to restart the kernel to use updated packages.


## Import


In [2]:
import os
from tqdm import tqdm
import time
import logging
import pandas as pd
import vk_api
from math import ceil
import vk
import pandas as pd
import os
import pickle
import pandas as pd
import pandas as pd
import vk_api
import time
from urllib.parse import urlparse
import pandas as pd
import glob
import os
import pandas as pd
import glob
import os
import json
import time
from datetime import datetime

## Поиск ID феминистических групп вк

In [7]:
# Read the access token from the file
with open('/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/TOKEN VK') as f:
    access_token = f.read().strip()

# Initialize VK session
vk_session = vk_api.VkApi(token=access_token)
vk = vk_session.get_api()

In [18]:
# Step 1: Read group IDs from the CSV file
#file_path = '/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/FEM groups VK.xlsx'
file_path = '/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/More info on the fem groups from the article.xlsx'
group_urls = pd.read_excel(file_path)['url']



In [19]:
group_ids = []

# Function to extract screen name from VK URL
def get_screen_name(url):
    if "vk.com/" in url:
        return url.split("vk.com/")[1]
    return None

In [20]:
# Iterate over each URL to get group ID
for url in group_urls:
    screen_name = get_screen_name(url)
    if screen_name:
        try:
            response = vk.utils.resolveScreenName(screen_name=screen_name)
            if response and response['type'] == 'group':
                group_ids.append(response['object_id'])
            else:
                group_ids.append(None)
        except vk_api.exceptions.ApiError as e:
            print(f"API error for {url}: {e}")
            group_ids.append(None)
    else:
        group_ids.append(None)

# Add group IDs to the DataFrame
#group_urls['group_id'] = group_ids

In [21]:
df = pd.DataFrame()

In [22]:
df['group_id'] = group_ids
df['url'] = group_urls

In [24]:
# Save the updated DataFrame if needed
df.to_excel('article FEM group ids (clened up).xlsx', index=False)


## Sorting FEM groups using VK API

In [4]:
# Read Excel file with URLs
df = pd.read_excel('/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/More info on the fem groups from the article.xlsx')  # Replace with your file path

In [5]:
# Read the access token from the file
with open('/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/TOKEN VK') as f:
    access_token = f.read().strip()

# Initialize VK session
vk_session = vk_api.VkApi(token=access_token)
vk = vk_session.get_api()

In [6]:
#def extract_group_id(url):
#    """Extract group ID or screen name from URL"""
#    path = urlparse(url).path
#    return path.strip('/')

def extract_group_id(url):
    """Extract group ID or screen name from URL"""
    path = urlparse(url).path
    # Remove leading/trailing slashes and 'public' or 'club' prefixes
    group_id = path.strip('/')
    if group_id.startswith('public'):
        group_id = group_id[6:]
    elif group_id.startswith('club'):
        group_id = group_id[4:]
    return group_id

In [13]:
def get_group_info(group_identifier):
    """Get group information using VK API"""
    try:
        group_info = vk.groups.getById(
            group_id=group_identifier,
            fields=['description', 'members_count']
        )[0]
        
        # Get wall posts
        posts = vk.wall.get(
            owner_id=f"-{group_info['id']}", 
            count=5
        )['items']
        
        # Extract post texts
        post_texts = []
        for post in posts:
            post_texts.append(post.get('text', ''))
        
        # Pad with None if less than 5 posts
        post_texts.extend([None] * (5 - len(post_texts)))
        
        return {
            'original_id': group_identifier,  # This is the ID/name from URL
            'name': group_info.get('name'),
            'description': group_info.get('description'),
            'members_count': group_info.get('members_count'),
            'post1': post_texts[0],
            'post2': post_texts[1],
            'post3': post_texts[2],
            'post4': post_texts[3],
            'post5': post_texts[4]
        }
    except vk_api.exceptions.ApiError as e:
        print(f"Error fetching group {group_identifier}: {str(e)}")
        return None

In [9]:
# Create lists to store the data
original_ids = []  # Store the original IDs from URLs
names = []
descriptions = []
member_counts = []
posts1 = []
posts2 = []
posts3 = []
posts4 = []
posts5 = []

In [14]:
# Process each URL
for url in df['url']:  # Replace 'url' with your column name
    group_id = extract_group_id(url)
    
    # Add delay to avoid hitting rate limits
    time.sleep(0.4)
    
    group_data = get_group_info(group_id)
    
    if group_data:
        original_ids.append(group_data['original_id'])
        names.append(group_data['name'])
        descriptions.append(group_data['description'])
        member_counts.append(group_data['members_count'])
        posts1.append(group_data['post1'])
        posts2.append(group_data['post2'])
        posts3.append(group_data['post3'])
        posts4.append(group_data['post4'])
        posts5.append(group_data['post5'])
    else:
        original_ids.append(None)
        names.append(None)
        descriptions.append(None)
        member_counts.append(None)
        posts1.append(None)
        posts2.append(None)
        posts3.append(None)
        posts4.append(None)
        posts5.append(None)


In [17]:
# Create results DataFrame
results_df = pd.DataFrame({
    'url': df['url'],
    'group_id': original_ids,
    'name': names,
    'description': descriptions,
    'member_count': member_counts,
    'post1': posts1,
    'post2': posts2,
    'post3': posts3,
    'post4': posts4,
    'post5': posts5
})

# Save to Excel
results_df.to_excel('More info on the fem groups from the article.xlsx', index=False)

## Pickle файлы чекаем

```all_groups_from_vk_100000001_100050000.pckl```  -- первые 5000 юзеров и группы, на которые они подписаны 


Читается Unpicke, получается словарь, в котором ключи -- это id пользователей, а значения -- группы, на которые они подписаны


In [8]:
import pickle
import pandas

In [13]:
#file_path = "/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/all_groups_from_vk_100000001_100050000.pckl"
file_path = 'all_groups_from_vk_100000001_100050000.pckl'
with open(file_path, 'rb') as file:
    unpickler = pickle.Unpickler(file)
    data = unpickler.load()
    

### Summary stat

In [14]:
# Initialize variables to track the largest length and corresponding key
max_len = 0
max_key = None

# Iterate through the dictionary
for key, value in data.items():
    if isinstance(value, (list, dict, set, tuple)):  # If the value is a collection
        length = len(value)
        if length > max_len:
            max_len = length
            max_key = key

# Output the key with the largest number of values and the length
print(f"Key with the most values: {max_key}, Number of values: {max_len}")


Key with the most values: 100008175, Number of values: 1000


In [15]:
from collections import defaultdict

# Dictionary to store how many keys each value is associated with
value_count = defaultdict(int)

# Iterate through the dictionary
for key, value in data.items():
    # If the value is a collection (list, dict, set, tuple), iterate through its elements
    if isinstance(value, (list, set, tuple)):
        for v in value:
            value_count[v] += 1
    else:
        value_count[value] += 1  # If it's a single value, count it directly

# Sort the values by frequency and get the top 5
top_5_common_values = sorted(value_count.items(), key=lambda x: x[1], reverse=True)[:5]

# Print the top 5 most common values and their counts
print("Top 5 most common values:")
for value, count in top_5_common_values:
    print(f"Value: {value}, Appears in {count} keys")


Top 5 most common values:
Value: None, Appears in 17548 keys
Value: 27895931, Appears in 950 keys
Value: 22798006, Appears in 923 keys
Value: 43215063, Appears in 877 keys
Value: 12382740, Appears in 769 keys


In [ ]:
data1 = data[100008175] #https://vk.com/id9828339
if 73730654 in data1: #https://vk.com/public114487519
    print(73730654)
print(data1)


73730654
(48512305, 90233697, 107854965, 26179938, 126100310, 34161144, 133987359, 75890438, 52364162, 22942343, 27615349, 104613976, 105999460, 30130071, 90937359, 35807071, 55621668, 62398004, 14395224, 10560742, 9884911, 18814693, 13684782, 17522135, 18789649, 16027476, 22096296, 19899031, 6925581, 9288126, 2064137, 22712274, 11787, 6453461, 6588892, 6836664, 7730502, 10389724, 10709065, 13201552, 14860717, 14947155, 15002520, 16054118, 16348810, 18835152, 19068382, 20517908, 21138656, 21328258, 22822623, 23007473, 23049237, 23164653, 23174180, 23337480, 23355020, 23433159, 23441339, 23470122, 23639186, 23758942, 23830580, 23891874, 24040594, 24220195, 24299135, 24399437, 24426108, 24458797, 24487105, 24489432, 24507410, 24551913, 24671775, 24706253, 24713873, 24720561, 24722253, 24847329, 24985591, 25025690, 25089415, 25094499, 25189523, 25199440, 25251165, 25379190, 25511407, 25539323, 25678919, 25682096, 25714310, 26002598, 26030283, 26158311, 26184323, 26188249, 26283323, 265384

## Looking for interesting group subs

In [2]:
import os
import pickle
import pandas as pd

In [2]:
# Load group IDs from the Excel file
#excel_file = 'Key_groups.xlsx'
excel_file = 'article FEM group ids (clened up).xlsx'

group_ids_of_interest = pd.read_excel(excel_file)['ID'].tolist()

In [3]:
# Initialize a dictionary to keep track of users for each group of interest
group_users = {group_id: [] for group_id in group_ids_of_interest}

In [4]:
# Function to filter users and track their group subscriptions with error handling
def track_users(file_path, group_users):
    try:
        with open(file_path, 'rb') as file:
            unpickler = pickle.Unpickler(file)
            data = unpickler.load()

        # Iterate through the data (user_id: tuple of group_ids)
        for user_id, group_ids in data.items():
            # Check if the user's group_ids is not None and is a valid iterable
            if group_ids is not None:
                # Check for each group of interest if the user is subscribed to it
                for group_id in group_ids_of_interest:
                    if group_id in group_ids:
                        group_users[group_id].append(user_id)

    except (pickle.UnpicklingError, EOFError, OSError) as e:
        print(f"Error processing file {file_path}: {e}")


In [5]:
# Directory containing all pickle files
pickle_dir = '/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/Pickles/collected_17_01_18'

# Get a list of all pickle files
pickle_files = [file for file in os.listdir(pickle_dir) if file.endswith('.pckl')]

# Use tqdm to display a progress bar while iterating through the files
for file_name in tqdm(pickle_files, desc='Processing pickle files'):
    file_path = os.path.join(pickle_dir, file_name)
    track_users(file_path, group_users)

Processing pickle files:   6%|▌         | 545/9268 [00:58<12:14, 11.88it/s]

Error processing file /Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/Pickles/collected_17_01_18/all_groups_from_vk_136000001_136050000.pckl: pickle data was truncated


Processing pickle files:  22%|██▏       | 2021/9268 [03:42<10:10, 11.87it/s]

Error processing file /Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/Pickles/collected_17_01_18/all_groups_from_vk_13600001_13650000.pckl: pickle data was truncated


Processing pickle files:  29%|██▉       | 2724/9268 [04:58<11:42,  9.32it/s]

...

Processing pickle files:  84%|████████▍ | 7819/9268 [17:06<02:05, 11.51it/s]

Error processing file /Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/Pickles/collected_17_01_18/all_groups_from_vk_326750001_326800000.pckl: pickle data was truncated


Processing pickle files:  90%|█████████ | 8385/9268 [18:09<01:08, 12.91it/s]

Error processing file /Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/Pickles/collected_17_01_18/all_groups_from_vk_292400001_292450000.pckl: pickle data was truncated


Processing pickle files: 100%|██████████| 9268/9268 [19:47<00:00,  7.81it/s]


In [6]:
# Print or save the result
for group_id, users in group_users.items():
    print(f"Group {group_id} has {len(users)} users subscribed.")

Group 81914161 has 6119 users subscribed.
Group 190884646 has 0 users subscribed.
Group 80728963 has 26309 users subscribed.
Group 153276634 has 978 users subscribed.
Group 96917976 has 18126 users subscribed.
Group 204732875 has 0 users subscribed.
Group 198043329 has 0 users subscribed.
Group 159437367 has 1989 users subscribed.
Group 74599974 has 6667 users subscribed.
Group 195772623 has 0 users subscribed.
Group 73197614 has 116 users subscribed.
Group 73837757 has 469 users subscribed.
Group 87819738 has 144 users subscribed.
Group 114487519 has 38 users subscribed.
Group 65878810 has 112 users subscribed.
Group 128501486 has 380 users subscribed.
Group 204172284 has 0 users subscribed.
Group 209222517 has 0 users subscribed.
Group 189672959 has 0 users subscribed.
Group 163233491 has 0 users subscribed.
Group 225770630 has 0 users subscribed.


In [7]:
# Optionally, save to a file
output_file = 'group_users_just_fem.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(group_users, f)


## Export

In [ ]:
file_path = 'group_users_25.10.24.pkl' #all the groups
with open(file_path, 'rb') as file:
    unpickler1 = pickle.Unpickler(file)
    group_users = unpickler1.load()

In [12]:
file_path = 'group_users_just_fem.pkl'
with open(file_path, 'rb') as file:
    unpickler1 = pickle.Unpickler(file)
    group_users = unpickler1.load()

In [13]:
# Create an empty DataFrame with user IDs as the index
user_ids = set()
for users in group_users.values():
    user_ids.update(users)

user_ids = list(user_ids)

# Create a DataFrame with user_ids as the index
df = pd.DataFrame(index=user_ids)

# Populate the DataFrame
for group_id, users in group_users.items():
    df[group_id] = df.index.isin(users)  # True if user is in the group, else False

# Optionally, you can fill NaN values with False (or keep them as is)
df = df.fillna(False)

# Display the resulting DataFrame
print(df)

           81914161   190884646  80728963   153276634  96917976   204732875  \
166461440      False      False      False      False       True      False   
7602177         True      False      False      False      False      False   
187826181      False      False       True      False      False      False   
62521349       False      False       True      False      False      False   
12320776       False      False       True      False      False      False   
...              ...        ...        ...        ...        ...        ...   
142213112      False      False      False      False       True      False   
143654905      False      False       True      False      False      False   
239861754      False      False      False      False      False      False   
64356349       False      False       True      False      False      False   
53215231       False      False       True      False      False      False   

           198043329  159437367  74599974   1957726

In [11]:
df.to_excel("user|group just FEM subs.xlsx")

# Getting the characteristics

In [ ]:
import pandas as pd
import glob
import os
import json
import time
from datetime import datetime

def update_user_characteristics(
    main_df, 
    csv_folder_path, 
    csv_id_column='id',  # Column name in CSVs
    chunk_size=100000,
    checkpoint_file='merge_progress.json',
    resume=True
):
    """
    Sequentially process CSV files and update main DataFrame that has user IDs as index.
    
    Parameters:
    main_df (pd.DataFrame): Your DataFrame with user IDs as index
    csv_folder_path (str): Path to folder containing CSV files
    csv_id_column (str): Name of the ID column in CSV files (default: 'id')
    chunk_size (int): Number of rows to process at once
    checkpoint_file (str): File to save progress
    resume (bool): Whether to resume from last checkpoint if available
    """
    def save_checkpoint(processed_files, remaining_ids, current_file=None, current_position=None):
        checkpoint = {
            'processed_files': list(processed_files),
            'remaining_ids': list(map(str, remaining_ids)),  # Convert IDs to strings for JSON
            'current_file': current_file,
            'current_position': current_position,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        with open(checkpoint_file, 'w') as f:
            json.dump(checkpoint, f)
    
    def load_checkpoint():
        try:
            with open(checkpoint_file, 'r') as f:
                checkpoint = json.load(f)
            return (
                set(checkpoint['processed_files']),
                set(map(str, checkpoint['remaining_ids'])),  # Convert back to same type as index
                checkpoint['current_file'],
                checkpoint['current_position']
            )
        except FileNotFoundError:
            return set(), set(map(str, main_df.index)), None, None
    
    # Initialize or resume progress
    if resume and os.path.exists(checkpoint_file):
        print("Found checkpoint file. Resuming from last position...")
        processed_files, remaining_ids, resume_file, resume_position = load_checkpoint()
        initial_user_count = len(remaining_ids) + (len(main_df) - len(remaining_ids))
        print(f"Resuming with {len(remaining_ids):,} users left to find")
    else:
        processed_files = set()
        remaining_ids = set(map(str, main_df.index))  # Convert index to strings for consistency
        initial_user_count = len(remaining_ids)
        resume_file = None
        resume_position = None
        print(f"Starting fresh search for {initial_user_count:,} users")
    
    # Get all CSV files
    csv_files = glob.glob(os.path.join(csv_folder_path, '*.csv'))
    if not csv_files:
        raise ValueError(f"No CSV files found in {csv_folder_path}")
    
    # Skip already processed files if resuming
    if resume_file:
        csv_files = [f for f in csv_files if f == resume_file or f not in processed_files]
    else:
        csv_files = [f for f in csv_files if f not in processed_files]
    
    # Process each file
    for file_path in csv_files:
        if not remaining_ids:
            print("\nAll users found! Stopping early.")
            break
            
        filename = os.path.basename(file_path)
        print(f"\nProcessing {filename}...")
        
        # First read one row to get column names
        sample_df = pd.read_csv(file_path, nrows=1)
        available_columns = set(sample_df.columns) - {csv_id_column}
        
        # Create chunk iterator
        chunks = pd.read_csv(
            file_path,
            chunksize=chunk_size,
            low_memory=False,
            dtype={csv_id_column: str}  # Ensure IDs are strings for consistent matching
        )
        
        rows_processed = 0
        matches_in_file = 0
        start_time = time.time()
        
        # Skip to resume position if applicable
        if resume_file == file_path and resume_position:
            skip_chunks = resume_position // chunk_size
            for _ in range(skip_chunks):
                next(chunks)
            rows_processed = skip_chunks * chunk_size
            print(f"Resumed from position {rows_processed:,}")
        
        # Process each chunk
        try:
            for chunk in chunks:
                # Filter only rows with IDs that exist in main_df
                matching_rows = chunk[chunk[csv_id_column].isin(remaining_ids)]
                
                if len(matching_rows) > 0:
                    # Ensure columns are present in main_df before updating
                    for col in available_columns:
                        if col not in main_df.columns:
                            main_df[col] = None
                    
                    # Align indices to prevent adding new rows
                    matching_rows = matching_rows.set_index(csv_id_column)
                    existing_users = matching_rows.index.intersection(main_df.index)
                    main_df.loc[existing_users, available_columns] = matching_rows.loc[existing_users, available_columns]

                    remaining_ids -= set(existing_users)
                    matches_in_file += len(existing_users)
                
                rows_processed += len(chunk)
                
                # Progress update and checkpoint save every million rows
                if rows_processed % (chunk_size * 10) == 0:
                    elapsed_time = time.time() - start_time
                    rate = rows_processed / elapsed_time if elapsed_time > 0 else 0
                    
                    print(f"Processed {rows_processed:,} rows ({rate:.0f} rows/sec)")
                    print(f"Found {matches_in_file:,} matches")
                    print(f"Still looking for {len(remaining_ids):,} users")
                    
                    # Save checkpoint
                    save_checkpoint(processed_files, remaining_ids, file_path, rows_processed)
            
            # Mark file as completely processed
            processed_files.add(file_path)
            save_checkpoint(processed_files, remaining_ids)
            
        except Exception as e:
            print(f"\nError processing file {filename}: {str(e)}")
            print("Progress saved. You can resume from this point.")
            save_checkpoint(processed_files, remaining_ids, file_path, rows_processed)
            raise
        
        # File summary
        print(f"\nFinished {filename}")
        print(f"Total rows processed: {rows_processed:,}")
        print(f"Matches found: {matches_in_file:,}")
        print(f"Users still to find: {len(remaining_ids):,}")
    
    # Final summary
    found_users = initial_user_count - len(remaining_ids)
    print(f"\nFinal Results:")
    print(f"Found {found_users:,} out of {initial_user_count:,} users")
    print(f"Added {len(available_columns)} characteristic columns")
    
    if remaining_ids:
        print(f"Could not find data for {len(remaining_ids):,} users")
    
    # Clean up checkpoint file if all done
    if os.path.exists(checkpoint_file) and not remaining_ids:
        os.remove(checkpoint_file)
        print("Processing complete - checkpoint file removed")
    
    return main_df


## func versions

In [ ]:
import pandas as pd
import glob
import os
import json
import time
from datetime import datetime

def update_user_characteristics(
    main_df, 
    csv_folder_path, 
    csv_id_column='id',  # Column name in CSVs
    chunk_size=100000,
    checkpoint_file='merge_progress.json',
    resume=True
):
    """
    Sequentially process CSV files and update main DataFrame that has user IDs as index.
    
    Parameters:
    main_df (pd.DataFrame): Your DataFrame with user IDs as index
    csv_folder_path (str): Path to folder containing CSV files
    csv_id_column (str): Name of the ID column in CSV files (default: 'id')
    chunk_size (int): Number of rows to process at once
    checkpoint_file (str): File to save progress
    resume (bool): Whether to resume from last checkpoint if available
    """
    def save_checkpoint(processed_files, remaining_ids, current_file=None, current_position=None):
        checkpoint = {
            'processed_files': list(processed_files),
            'remaining_ids': list(map(str, remaining_ids)),  # Convert IDs to strings for JSON
            'current_file': current_file,
            'current_position': current_position,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        with open(checkpoint_file, 'w') as f:
            json.dump(checkpoint, f)
    
    def load_checkpoint():
        try:
            with open(checkpoint_file, 'r') as f:
                checkpoint = json.load(f)
            return (
                set(checkpoint['processed_files']),
                set(map(str, checkpoint['remaining_ids'])),  # Convert back to same type as index
                checkpoint['current_file'],
                checkpoint['current_position']
            )
        except FileNotFoundError:
            return set(), set(map(str, main_df.index)), None, None
    
    # Initialize or resume progress
    if resume and os.path.exists(checkpoint_file):
        print("Found checkpoint file. Resuming from last position...")
        processed_files, remaining_ids, resume_file, resume_position = load_checkpoint()
        initial_user_count = len(remaining_ids) + (len(main_df) - len(remaining_ids))
        print(f"Resuming with {len(remaining_ids):,} users left to find")
    else:
        processed_files = set()
        remaining_ids = set(map(str, main_df.index))  # Convert index to strings for consistency
        initial_user_count = len(remaining_ids)
        resume_file = None
        resume_position = None
        print(f"Starting fresh search for {initial_user_count:,} users")
    
    # Get all CSV files
    csv_files = glob.glob(os.path.join(csv_folder_path, '*.csv'))
    if not csv_files:
        raise ValueError(f"No CSV files found in {csv_folder_path}")
    
    # Skip already processed files if resuming
    if resume_file:
        csv_files = [f for f in csv_files if f == resume_file or f not in processed_files]
    else:
        csv_files = [f for f in csv_files if f not in processed_files]
    
    # Process each file
    for file_path in csv_files:
        if not remaining_ids:
            print("\nAll users found! Stopping early.")
            break
            
        filename = os.path.basename(file_path)
        print(f"\nProcessing {filename}...")
        
        # First read one row to get column names
        sample_df = pd.read_csv(file_path, nrows=1)
        available_columns = set(sample_df.columns) - {csv_id_column}
        
        # Create chunk iterator
        chunks = pd.read_csv(
            file_path,
            chunksize=chunk_size,
            low_memory=False,
            dtype={csv_id_column: str}  # Ensure IDs are strings for consistent matching
        )
        
        rows_processed = 0
        matches_in_file = 0
        start_time = time.time()
        
        # Skip to resume position if applicable
        if resume_file == file_path and resume_position:
            skip_chunks = resume_position // chunk_size
            for _ in range(skip_chunks):
                next(chunks)
            rows_processed = skip_chunks * chunk_size
            print(f"Resumed from position {rows_processed:,}")
        
        # Process each chunk
        try:
            for chunk in chunks:
                # Find matching users in this chunk
                matching_mask = chunk[csv_id_column].isin(remaining_ids)
                matching_rows = chunk[matching_mask]
                
                if len(matching_rows) > 0:
                    # Update main DataFrame with found users
                    for _, row in matching_rows.iterrows():
                        user_id = str(row[csv_id_column])  # Ensure string type
                        
                        # Update each column for this user
                        for col in available_columns:
                            if col not in main_df.columns:
                                main_df[col] = None
                            main_df.loc[user_id, col] = row[col]
                        
                        remaining_ids.remove(user_id)
                    
                    matches_in_file += len(matching_rows)
                
                rows_processed += len(chunk)
                
                # Progress update and checkpoint save every million rows
                if rows_processed % (chunk_size * 10) == 0:
                    elapsed_time = time.time() - start_time
                    rate = rows_processed / elapsed_time if elapsed_time > 0 else 0
                    
                    print(f"Processed {rows_processed:,} rows ({rate:.0f} rows/sec)")
                    print(f"Found {matches_in_file:,} matches")
                    print(f"Still looking for {len(remaining_ids):,} users")
                    
                    # Save checkpoint
                    save_checkpoint(processed_files, remaining_ids, file_path, rows_processed)
            
            # Mark file as completely processed
            processed_files.add(file_path)
            save_checkpoint(processed_files, remaining_ids)
            
        except Exception as e:
            print(f"\nError processing file {filename}: {str(e)}")
            print("Progress saved. You can resume from this point.")
            save_checkpoint(processed_files, remaining_ids, file_path, rows_processed)
            raise
        
        # File summary
        print(f"\nFinished {filename}")
        print(f"Total rows processed: {rows_processed:,}")
        print(f"Matches found: {matches_in_file:,}")
        print(f"Users still to find: {len(remaining_ids):,}")
    
    # Final summary
    found_users = initial_user_count - len(remaining_ids)
    print(f"\nFinal Results:")
    print(f"Found {found_users:,} out of {initial_user_count:,} users")
    print(f"Added {len(available_columns)} characteristic columns")
    
    if remaining_ids:
        print(f"Could not find data for {len(remaining_ids):,} users")
    
    # Clean up checkpoint file if all done
    if os.path.exists(checkpoint_file) and not remaining_ids:
        os.remove(checkpoint_file)
        print("Processing complete - checkpoint file removed")
    
    return main_df

# Example usage:
"""
# Assuming df has user IDs as index:
update_user_characteristics(
    main_df=df,
    csv_folder_path='your_csv_folder',
    csv_id_column='id',  # The column name in your CSV files
    chunk_size=100000,
    checkpoint_file='merge_progress.json',
    resume=True
)
"""

In [ ]:
import pandas as pd
import glob
import os

def merge_multiple_user_characteristics(existing_df, csv_folder_path, user_id_column='user_id'):
    """
    Memory efficiently merges user characteristics from multiple CSV files into an existing DataFrame
    by processing one file at a time.
    
    Parameters:
    existing_df (pandas.DataFrame): Your existing DataFrame with user IDs in the index
    csv_folder_path (str): Path to the folder containing CSV files with user characteristics
    user_id_column (str): Name of the user ID column in characteristic files (default: 'user_id')
    
    Returns:
    pandas.DataFrame: Merged DataFrame with added user characteristics
    """
    # Get list of all CSV files in the specified folder
    csv_files = glob.glob(os.path.join(csv_folder_path, '*.csv'))
    
    if not csv_files:
        raise ValueError(f"No CSV files found in {csv_folder_path}")
    
    # Make a copy of the existing DataFrame to add characteristics to
    result_df = existing_df.copy()
    
    # Get the set of user IDs we're interested in
    existing_user_ids = set(existing_df.index)
    
    # Process each CSV file individually
    for file_path in csv_files:
        try:
            # Read the CSV file
            df = pd.read_csv(file_path)
            
            # Verify that the user ID column exists in this file
            if user_id_column not in df.columns:
                print(f"Warning: '{user_id_column}' not found in {os.path.basename(file_path)}. Skipping file.")
                continue
            
            # Set the index to user_id for easier merging
            df.set_index(user_id_column, inplace=True)
            
            # Only keep rows for existing users
            df = df[df.index.isin(existing_user_ids)]
            
            if len(df) > 0:
                # Add new columns to result_df
                new_columns = [col for col in df.columns if col not in result_df.columns]
                for col in new_columns:
                    result_df[col] = None
                
                # Update values for matched users
                result_df.update(df)
                
                print(f"Successfully processed {os.path.basename(file_path)}: "
                      f"matched {len(df)} users, added {len(new_columns)} new characteristics")
            else:
                print(f"No matching users found in {os.path.basename(file_path)}")
            
        except Exception as e:
            print(f"Error reading {os.path.basename(file_path)}: {str(e)}")
            continue
    
    # Report final results
    final_new_columns = set(result_df.columns) - set(existing_df.columns)
    print(f"\nFinal results:")
    print(f"Added {len(final_new_columns)} new characteristic columns")
    print(f"Final DataFrame shape: {result_df.shape}")
    
    return result_df

In [11]:
import pandas as pd
import glob
import os

def merge_multiple_user_characteristics(existing_df, csv_folder_path, user_id_column='user_id'):
    """
    Merges user characteristics from multiple CSV files into an existing DataFrame.
    Only adds characteristics for existing users without creating duplicates.
    
    Parameters:
    existing_df (pandas.DataFrame): Your existing DataFrame with user IDs
    csv_folder_path (str): Path to the folder containing CSV files with user characteristics
    user_id_column (str): Name of the user ID column (default: 'user_id')
    
    Returns:
    pandas.DataFrame: Merged DataFrame with added user characteristics
    """
    # Reset index if it contains the user IDs to ensure proper merging
    if existing_df.index.name == user_id_column:
        existing_df = existing_df.reset_index()
    
    # Get list of all CSV files in the specified folder
    csv_files = glob.glob(os.path.join(csv_folder_path, '*.csv'))
    
    if not csv_files:
        raise ValueError(f"No CSV files found in {csv_folder_path}")
    
    # Initialize an empty list to store all characteristic DataFrames
    characteristic_dfs = []
    
    # Read and process each CSV file
    for file_path in csv_files:
        try:
            df = pd.read_csv(file_path)
            
            # Verify that the user ID column exists in this file
            if user_id_column not in df.columns:
                print(f"Warning: '{user_id_column}' not found in {os.path.basename(file_path)}. Skipping file.")
                continue
            
            # Only keep rows where user_id exists in the existing_df
            df = df[df[user_id_column].isin(existing_df[user_id_column])]
            
            characteristic_dfs.append(df)
            print(f"Successfully read {os.path.basename(file_path)}: {len(df)} rows")
            
        except Exception as e:
            print(f"Error reading {os.path.basename(file_path)}: {str(e)}")
            continue
    
    if not characteristic_dfs:
        raise ValueError("No valid CSV files could be processed")
    
    # Combine all characteristic DataFrames
    print("\nCombining characteristics from all files...")
    combined_characteristics = pd.concat(characteristic_dfs, ignore_index=True)
    
    # Remove any duplicate user entries (keep the last occurrence)
    if combined_characteristics.duplicated(subset=user_id_column).any():
        duplicate_count = combined_characteristics.duplicated(subset=user_id_column).sum()
        print(f"Found {duplicate_count} duplicate user entries. Keeping most recent entries.")
        combined_characteristics = combined_characteristics.drop_duplicates(
            subset=user_id_column, 
            keep='last'
        )
    
    # Merge with the existing DataFrame
    print("\nMerging with existing DataFrame...")
    merged_df = existing_df.merge(
        combined_characteristics,
        on=user_id_column,
        how='left'
    )
    
    # Additional verification
    assert len(merged_df) == len(existing_df), "Merged DataFrame has a different number of rows than the original"
    
    # Report on merge results
    unmatched_users = existing_df[~existing_df[user_id_column].isin(combined_characteristics[user_id_column])]
    if len(unmatched_users) > 0:
        print(f"\nWarning: {len(unmatched_users)} users from existing DataFrame not found in characteristics files")
    
    print(f"\nFinal merged DataFrame shape: {merged_df.shape}")
    new_columns = set(merged_df.columns) - set(existing_df.columns)
    print(f"Added {len(new_columns)} new characteristic columns")
    
    return merged_df

In [14]:
# 1. First, make sure you have your initial DataFrame ready
# Let's say your initial df has user IDs either as a column or as the index
# and some other columns (like group IDs)
initial_df = df  # your existing DataFrame

# 2. Know your folder path where the CSV files with characteristics are stored
csv_folder = '/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/Andrey Simonov/useful users'  # replace with your actual folder path

# 3. Know what your user ID column is called
# If your user IDs are in a column, use that column name
user_id_name = "id"  # replace with your actual user ID column name
# OR if your user IDs are in the index, use the index name
user_id_name = initial_df.index.name

# 4. Call the function
enriched_df = merge_multiple_user_characteristics(
    existing_df=initial_df,
    csv_folder_path=csv_folder,
    user_id_column=user_id_name
)

# 5. The enriched_df will now have all the characteristics merged in
# You can check what new columns were added:
new_columns = set(enriched_df.columns) - set(initial_df.columns)
print("New columns added:", new_columns)

/var/folders/fj/5vkqx46j2sqfv58rcgb2wynr0000gn/T/ipykernel_1498/3182338328.py:34: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


KeyboardInterrupt: 

## Using one of the functions

In [10]:

#update_user_characteristics(
#    main_df=df,
#    csv_folder_path='/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/Andrey Simonov/useful users',
#    user_id_column='id',
#    chunk_size=100000,
#    checkpoint_file='merge_progress.json',  # Specify checkpoint file location
#    resume=True  # Set to False to start fresh
#)
update_user_characteristics(
    main_df=df,
    csv_folder_path='/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/Andrey Simonov/useful users',
    csv_id_column='id',  # The column name in CSV files
    chunk_size=100000
)

Starting fresh search for 57,097 users

Processing vk_users_data_430_sframe_part9.csv...
Processed 1,000,000 rows (321658 rows/sec)
Found 141 matches
Still looking for 56,956 users
Processed 2,000,000 rows (329892 rows/sec)
Found 270 matches
Still looking for 56,827 users
Processed 3,000,000 rows (326055 rows/sec)
Found 405 matches
Still looking for 56,692 users
Processed 4,000,000 rows (327423 rows/sec)
Found 533 matches
Still looking for 56,564 users
Processed 5,000,000 rows (328978 rows/sec)
Found 650 matches
Still looking for 56,447 users
Processed 6,000,000 rows (331601 rows/sec)
Found 764 matches
Still looking for 56,333 users
Processed 7,000,000 rows (331221 rows/sec)
Found 889 matches
Still looking for 56,208 users
Processed 8,000,000 rows (329731 rows/sec)
Found 1,009 matches
Still looking for 56,088 users
Processed 9,000,000 rows (332578 rows/sec)
Found 1,115 matches
Still looking for 55,982 users
Processed 10,000,000 rows (334226 rows/sec)
Found 1,209 matches
Still looking f

,81914161,190884646,80728963,153276634,96917976,204732875,198043329,159437367,74599974,195772623,...,sex,instagram,personal.political,personal.alcohol,personal.smoking,personal.life_main,personal.inspired_by,personal.people_main,personal.langs,personal.religion
166461440,False,False,False,False,True,False,False,False,False,False,...,None,None,None,None,None,None,None,None,None,None
7602177,True,False,False,False,False,False,False,False,False,False,...,None,None,None,None,None,None,None,None,None,None
187826181,False,False,True,False,False,False,False,False,False,False,...,None,None,None,None,None,None,None,None,None,None
62521349,False,False,True,False,False,False,False,False,False,False,...,None,None,None,None,None,None,None,None,None,None
12320776,False,False,True,False,False,False,False,False,False,False,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430136478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430146222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430147201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430167164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,sofiyalekseeva,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# GPT

In [ ]:
import pandas as pd
import glob
import os
import json
import time
from datetime import datetime

def update_user_characteristics(
    main_df, 
    filename, 
    csv_id_column='id', 
    remaining_ids # Column name in CSVs
):
    """
    Sequentially process CSV files and update main DataFrame that has user IDs as index.
    
    Parameters:
    main_df (pd.DataFrame): Your DataFrame with user IDs as index
    csv_folder_path (str): Path to folder containing CSV files
    csv_id_column (str): Name of the ID column in CSV files (default: 'id')
    chunk_size (int): Number of rows to process at once
    checkpoint_file (str): File to save progress
    resume (bool): Whether to resume from last checkpoint if available
    """

    


    processed_files = set()
  # Convert index to strings for consistency
    initial_user_count = len(remaining_ids)
    resume_file = None
    resume_position = None
    print(f"Starting fresh search for {initial_user_count:,} users")

    if not remaining_ids:
        print("\nAll users found! Stopping early.")
        return (pd.DataFrame())
            
    filename = os.path.basename(file_path)
    print(f"\nProcessing {filename}...")
    
    # First read one row to get column names
    sample_df = pd.read_csv(file_path, nrows=1)
    available_columns = set(sample_df.columns) - {csv_id_column}
    
    # Create chunk iterator
    chunks = pd.read_csv(
        file_path,
        chunksize=chunk_size,
        low_memory=False,
        dtype={csv_id_column: str}  # Ensure IDs are strings for consistent matching
    )
    
    rows_processed = 0
    matches_in_file = 0
    start_time = time.time()
    
    # Skip to resume position if applicable
    if resume_file == file_path and resume_position:
        skip_chunks = resume_position // chunk_size
        for _ in range(skip_chunks):
            next(chunks)
        rows_processed = skip_chunks * chunk_size
        print(f"Resumed from position {rows_processed:,}")
    
    # Process each chunk
    try:
        for chunk in chunks:
            # Filter only rows with IDs that exist in main_df
            matching_rows = chunk[chunk[csv_id_column].isin(remaining_ids)]
            
            if len(matching_rows) > 0:
                # Ensure columns are present in main_df before updating
                for col in available_columns:
                    if col not in main_df.columns:
                        main_df[col] = None
                
                # Align indices to prevent adding new rows
                # Align indices to prevent adding new rows
                matching_rows = matching_rows.set_index(csv_id_column)
                existing_users = list(matching_rows.index.intersection(main_df.index))  # Convert to list
                available_columns = list(available_columns)  # Ensure available_columns is a list
                main_df.loc[existing_users, available_columns] = matching_rows.loc[existing_users, available_columns]

                # Update remaining_ids to remove found users (convert to list for consistency)
                remaining_ids -= set(existing_users)


                #matching_rows = matching_rows.set_index(csv_id_column)
                #existing_users = matching_rows.index.intersection(main_df.index)
                #main_df.loc[existing_users, available_columns] = matching_rows.loc[existing_users, available_columns]

                #remaining_ids -= set(existing_users)
                matches_in_file += len(existing_users)
            
            rows_processed += len(chunk)
            
            # Progress update and checkpoint save every million rows
            if rows_processed % (chunk_size * 10) == 0:
                elapsed_time = time.time() - start_time
                rate = rows_processed / elapsed_time if elapsed_time > 0 else 0
                
                print(f"Processed {rows_processed:,} rows ({rate:.0f} rows/sec)")
                print(f"Found {matches_in_file:,} matches")
                print(f"Still looking for {len(remaining_ids):,} users")
                
                # Save checkpoint
                save_checkpoint(processed_files, remaining_ids, file_path, rows_processed)
        
        # Mark file as completely processed
        processed_files.add(file_path)
        save_checkpoint(processed_files, remaining_ids)
        
    except Exception as e:
        print(f"\nError processing file {filename}: {str(e)}")
        print("Progress saved. You can resume from this point.")
        save_checkpoint(processed_files, remaining_ids, file_path, rows_processed)
        raise
    
    # File summary
    print(f"\nFinished {filename}")
    print(f"Total rows processed: {rows_processed:,}")
    print(f"Matches found: {matches_in_file:,}")
    print(f"Users still to find: {len(remaining_ids):,}")
    
    # Final summary
    found_users = initial_user_count - len(remaining_ids)
    print(f"\nFinal Results:")
    print(f"Found {found_users:,} out of {initial_user_count:,} users")
    print(f"Added {len(available_columns)} characteristic columns")
    
    if remaining_ids:
        print(f"Could not find data for {len(remaining_ids):,} users")
    
    # Clean up checkpoint file if all done
    if os.path.exists(checkpoint_file) and not remaining_ids:
        os.remove(checkpoint_file)
        print("Processing complete - checkpoint file removed")
    
    return main_df


In [ ]:
remaining_ids = set(map(str, df.index))
# Assuming df is your initial DataFrame with user IDs as index
for filename in filenames:
    print(f"Processing &s file" %filename)
    updated_df, upd_remaining_ids = update_user_characteristics(
        main_df=df,
        filename = filename,  
        remaining_ids = remaining_ids# Adjust chunk size if needed
    )
    print("Found", len(remaining_ids) - len(upd_remaining_ids))
    df = updated_df
    remaining_ids = upd_remaining_ids

Starting fresh search for 57,097 users

Processing vk_users_data_430_sframe_part9.csv...
Processed 1,000,000 rows (736901 rows/sec)
Found 0 matches
Still looking for 57,097 users
Processed 2,000,000 rows (745883 rows/sec)
Found 0 matches
Still looking for 57,097 users
Processed 3,000,000 rows (750227 rows/sec)
Found 0 matches
Still looking for 57,097 users
Processed 4,000,000 rows (741226 rows/sec)
Found 0 matches
Still looking for 57,097 users
Processed 5,000,000 rows (738537 rows/sec)
Found 0 matches
Still looking for 57,097 users
Processed 6,000,000 rows (735934 rows/sec)
Found 0 matches
Still looking for 57,097 users
Processed 7,000,000 rows (729094 rows/sec)
Found 0 matches
Still looking for 57,097 users
Processed 8,000,000 rows (728933 rows/sec)
Found 0 matches
Still looking for 57,097 users
Processed 9,000,000 rows (715061 rows/sec)
Found 0 matches
Still looking for 57,097 users
Processed 10,000,000 rows (710505 rows/sec)
Found 0 matches
Still looking for 57,097 users
Processed 

In [ ]:
df.to_excel("user|group just FEM subs.xlsx")

# DUMP


In [ ]:
import pickle

def load_pckl_in_batches(file_path, batch_size):
    observations = []
    count = 0
    with open(file_path, 'rb') as file:
        unpickler = pickle.Unpickler(file)
        while count < batch_size:
            #if count % 100 == 0:
                #print(count)
            try:
                #last_name	twitter	sex	id	home_town	first_name	bdate	city.id	city.title	personal.people_main	personal.alcohol	personal.inspired_by	personal.political	personal.religion	personal.smoking	personal.langs	personal.life_main	country.id	country.title	facebook_name	facebook	livejournal
                
                data = unpickler.load()
                print(type(data), count)
                #filtered_data = filter_keys(data, keys_to_keep)
                #observations.append(filtered_data)
                #observations.append(unpickler.load())
                if isinstance(data, list):
                    # Flatten the list by filtering necessary keys for each dictionary
                    for data in data:
                        filtered_data = filter_keys(data, keys_to_keep)
                        observations.append(filtered_data)
                        count += 1
            except EOFError:
                print("End of file reached.")
                break
            except pickle.UnpicklingError:
                # Data corruption detected, try to recover by skipping ahead
                print("Warning: UnpicklingError detected, skipping ahead in the file.")
                file.seek(file.tell() + 1024)  # Skip 1KB forward and try again
                continue
    return observations

# Example: Load in batches of 5 records
file_path = "/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/all_groups_from_vk_100000001_100050000.pckl"
with open(file_path, 'rb') as file:
    unpickler = pickle.Unpickler(file)
batch_size = 5

# Load the first batch
batch_data = load_pckl_in_batches(file_path, batch_size)

# Convert the list of dictionaries into a DataFrame
#df = pd.DataFrame(batch_data)
# Step 1: Flatten the outer list (list of lists of dictionaries)
#flat_data = [item for sublist in batch_data for item in sublist]


